In [4]:
!pip3 install flask-migrate


  Using cached Flask_Migrate-4.1.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
Using cached Flask_Migrate-4.1.0-py3-none-any.whl (21 kB)
Using cached alembic-1.15.2-py3-none-any.whl (231 kB)
Using cached mako-1.3.10-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3.13 -m pip install --upgrade pip


In [36]:
import sys
import os

# Path to your app root so imports work
sys.path.append(os.path.abspath('.'))

from app.extensions import db
from app.core.models import EventTeamRequirement, Event



In [37]:
from app import create_app
app = create_app()

# Set up the app context for DB access
app.app_context().push()


In [38]:
db.session.query(EventTeamRequirement).count()


218

In [39]:
for e in db.session.query(Event).order_by(Event.date.desc()).limit(5):
    print(f"{e.id}: {e.name} on {e.date.date()}")


4: Event Next Year on 2026-03-08
14: Sunday Service on 2025-09-14
10: Sunday Service on 2025-08-10
11: Worship Night on 2025-07-13
12: Sunday Service on 2025-07-06


In [40]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to the SQLite database
engine = create_engine('sqlite:///instance/church.db')

# Example query: Get all volunteers
query = "SELECT * FROM Volunteer"
df = pd.read_sql_query(query, engine)

# Display the data
df.head()


OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [41]:
import os
os.listdir("instance")  # See if church.db is visible


FileNotFoundError: [Errno 2] No such file or directory: 'instance'

In [42]:
from sqlalchemy import create_engine
import pandas as pd

# Replace with your actual path
db_path = "/Users/developer/Projects/church_scheduler_app/instance/church.db"

# Connect
engine = create_engine(f"sqlite:///{db_path}")

# Try a query
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", engine)


,name
0,volunteer
1,team
2,volunteer_team
3,role
4,volunteer_team_role
5,alembic_version
6,event_template
7,event_team_requirement
8,event
9,team_role


In [43]:
query = """
SELECT 
    m.name AS table_name,
    p.name AS column_name,
    p.type AS data_type,
    p.[notnull] AS not_null,
    p.[dflt_value] AS default_value,
    p.[pk] AS primary_key
FROM sqlite_master m
JOIN pragma_table_info(m.name) p
WHERE m.type = 'table'
ORDER BY m.name, p.cid
"""

schema_df = pd.read_sql_query(query, engine)
schema_df


,table_name,column_name,data_type,not_null,default_value,primary_key
0,alembic_version,version_num,VARCHAR(32),1,None,1
1,event,id,INTEGER,1,None,1
2,event,name,VARCHAR(100),1,None,0
3,event,date,DATETIME,1,None,0
4,event,description,TEXT,0,None,0
...,...,...,...,...,...,...
60,volunteer_team_role,id,INTEGER,1,None,1
61,volunteer_team_role,volunteer_id,INTEGER,0,None,0
62,volunteer_team_role,team_id,INTEGER,0,None,0
63,volunteer_team_role,role_id,INTEGER,0,None,0


In [46]:
from sqlalchemy import text

# First, get schema info
schema_query = """
SELECT 
    m.name AS table_name,
    p.name AS column_name,
    p.type AS data_type,
    p.[notnull] AS not_null,
    p.[dflt_value] AS default_value,
    p.[pk] AS primary_key
FROM sqlite_master m
JOIN pragma_table_info(m.name) p
WHERE m.type = 'table'
ORDER BY m.name, p.cid
"""

schema_df = pd.read_sql_query(schema_query, engine)

# Then, get row counts per table
tables = schema_df['table_name'].unique()
row_counts = {
    table: pd.read_sql_query(f"SELECT COUNT(*) AS count FROM {table}", engine).iloc[0]['count']
    for table in tables
}

# Merge row counts into schema DataFrame
schema_df['row_count'] = schema_df['table_name'].map(row_counts)

schema_df


,table_name,column_name,data_type,not_null,default_value,primary_key,row_count
0,alembic_version,version_num,VARCHAR(32),1,None,1,1
1,event,id,INTEGER,1,None,1,17
2,event,name,VARCHAR(100),1,None,0,17
3,event,date,DATETIME,1,None,0,17
4,event,description,TEXT,0,None,0,17
...,...,...,...,...,...,...,...
60,volunteer_team_role,id,INTEGER,1,None,1,27
61,volunteer_team_role,volunteer_id,INTEGER,0,None,0,27
62,volunteer_team_role,team_id,INTEGER,0,None,0,27
63,volunteer_team_role,role_id,INTEGER,0,None,0,27


In [45]:
from sqlalchemy import text

# First, get schema info
schema_query = """
SELECT 
    m.name AS table_name,
    p.name AS column_name,
    p.type AS data_type,
    p.[notnull] AS not_null,
    p.[dflt_value] AS default_value,
    p.[pk] AS primary_key
FROM sqlite_master m
JOIN pragma_table_info(m.name) p
WHERE m.type = 'table'
ORDER BY m.name, p.cid
"""

schema_df = pd.read_sql_query(schema_query, engine)

# Then, get row counts per table
tables = schema_df['table_name'].unique()
row_counts = {
    table: pd.read_sql_query(f"SELECT COUNT(*) AS count FROM {table}", engine).iloc[0]['count']
    for table in tables
}

# Merge row counts into schema DataFrame
schema_df['row_count'] = schema_df['table_name'].map(row_counts)




In [47]:
schema_df.to_csv("/Users/developer/Documents/db_schema_with_counts.csv", index=False)

